# Marcin Łukaszyk WUM PD 2

## Kodowanie zmiennych kategorycznych

W pierwszej części zadaniem jest zakodowanie zmiennych kategorycznych.Najpierw użyjemy Target Encoding a potem użyjemy algorytmu One Hot jak i Polynomial Coding oraz Helmert Coding.
Będziemy pracować na zborze danych "Allegro".


## Sposoby działania algorytmów

### One Hot:

Ten algorytm dla każdej zmiennej przyporządkowuje jedną kolumne dla jednej zmiennej konkretnego typy

### Polynomial Coding:

Ten algorytm "szuka" linowych,kwadratowych i sześćściennych zależnośći pomiędzy zmiennymi kategorycznymi i na tej podstawie koduje te zmienne 


### Helmert Coding:
Ten algorytm koduje zmienne na podstawie średnich tych zmiennych które jeszcze nie zostały zakodowane

## KOD

### Wczytanie Danych I Importy

In [119]:
import pandas as pd
import category_encoders as ce
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from math import sqrt
import copy
import warnings

warnings.filterwarnings('ignore')

data = pd.read_csv('allegro-api-transactions.csv')
pd.set_option('display.max_columns', 8)

### Czy są braki danych?

In [9]:
na_ratio_cols = data.isna().mean(axis=0)
na_ratio_cols


lp                        0.0
date                      0.0
item_id                   0.0
categories                0.0
pay_option_on_delivery    0.0
pay_option_transfer       0.0
seller                    0.0
price                     0.0
it_is_allegro_standard    0.0
it_quantity               0.0
it_is_brand_zone          0.0
it_seller_rating          0.0
it_location               0.0
main_category             0.0
dtype: float64

Nie ma braków danych. To dobrze :)

## Target Encoding

In [28]:
target_v = data['price'].to_numpy()
values = data['it_location'].to_numpy()
values_main = data['main_category'].to_numpy()

te = ce.target_encoder.TargetEncoder(data)

data_tmp = data.copy()
encoded = te.fit_transform(data['it_location'], data['price'])
data_tmp['target_encoded'] = encoded

print(data_tmp[['it_location', 'price', 'target_encoded']])

                it_location   price  target_encoded
0                  Warszawa   59.99       85.423398
1                  Warszawa    4.90       85.423398
2                    Leszno  109.90       61.990914
3       Wola Krzysztoporska   18.50       35.433365
4                 BIAŁYSTOK   19.90      117.191956
...                     ...     ...             ...
420015              Kraśnik  180.00       24.306929
420016          Dzierżoniów   14.99       66.785334
420017              Supraśl    5.99       18.682800
420018               Poznań  200.00      106.203076
420019                Pszów  500.00       78.136792

[420020 rows x 3 columns]


## One Hot

In [22]:
te = ce.one_hot.OneHotEncoder(data)

data_tmp = data.copy()
encoded = te.fit_transform(data['main_category'])
encoded = encoded.head(8)
print(encoded)

   main_category_1  main_category_2  main_category_3  main_category_4  ...  \
0                1                0                0                0  ...   
1                0                1                0                0  ...   
2                0                0                1                0  ...   
3                0                0                0                1  ...   
4                0                1                0                0  ...   
5                0                1                0                0  ...   
6                0                1                0                0  ...   
7                0                0                0                0  ...   

   main_category_24  main_category_25  main_category_26  main_category_27  
0                 0                 0                 0                 0  
1                 0                 0                 0                 0  
2                 0                 0                 0              

Każda zmienna ma jedynke w osobnym miejscu.Widać że dla rzędu 1 oraz 4 jedynka występuje w tej samej kolumnie co oznacza żę kodują one taką samą zmienną.

## Polynomial Coding

In [25]:
te = ce.polynomial.PolynomialEncoder(data)

data_tmp = data.copy()
encoded = te.fit_transform(data['main_category'])
print(encoded.head(8))

   intercept  main_category_0  main_category_1  main_category_2  ...  \
0          1        -0.321208         0.385019        -0.407465  ...   
1          1        -0.296500         0.296168        -0.219404  ...   
2          1        -0.271791         0.214426        -0.068956  ...   
3          1        -0.247083         0.139791         0.047015  ...   
4          1        -0.296500         0.296168        -0.219404  ...   
5          1        -0.296500         0.296168        -0.219404  ...   
6          1        -0.296500         0.296168        -0.219404  ...   
7          1        -0.222375         0.072265         0.131643  ...   

   main_category_22  main_category_23  main_category_24  main_category_25  
0         -0.000006          0.000002     -3.206859e-07      4.490496e-08  
1          0.000131         -0.000035      7.696462e-06     -1.167529e-06  
2         -0.001243          0.000370     -8.818862e-05      1.459412e-05  
3          0.007334         -0.002432      6.41

## Helmert Coding

In [26]:
te = ce.helmert.HelmertEncoder(data)

data_tmp = data.copy()
encoded = te.fit_transform(data['main_category'])
print(encoded.head(8))

   intercept  main_category_0  main_category_1  main_category_2  ...  \
0          1             -1.0             -1.0             -1.0  ...   
1          1              1.0             -1.0             -1.0  ...   
2          1              0.0              2.0             -1.0  ...   
3          1              0.0              0.0              3.0  ...   
4          1              1.0             -1.0             -1.0  ...   
5          1              1.0             -1.0             -1.0  ...   
6          1              1.0             -1.0             -1.0  ...   
7          1              0.0              0.0              0.0  ...   

   main_category_22  main_category_23  main_category_24  main_category_25  
0              -1.0              -1.0              -1.0              -1.0  
1              -1.0              -1.0              -1.0              -1.0  
2              -1.0              -1.0              -1.0              -1.0  
3              -1.0              -1.0          

### One Hot a Target Encoding?

Jedną z zalet Target Encodingu jest nie zajmowanie dodatkowego miejsca gdzie przy One-Hot zajmujemy go proporcjonalnie do ilości rózny wartośći.Wadą natomiast jest możliwość data leak. Tzn że przez małą ilośći danych o konkretej zmiennej algorytm nie będzie działał poprawnie.

# Uzupełnianie Braków Danych

Z powyższego zbioru danych usuniemy 10% wartości z kolumny "it_seller_rating".Nastęnie spróbujemy algorytmem  Nearest neighbors imputation uzupełnić te braki danych

In [120]:
n=int(420020/10) #Wielkość Danych
nan = np.nan
data2 = copy.deepcopy(data)
data3 = copy.deepcopy(data)
data2 = data2.head(n)
data2 = data2[['price', 'it_quantity']]
row = data3['it_seller_rating']
row_comp = copy.deepcopy(row)

for i in range(int(n/10)):
    row.loc[i * 10] = nan
data2['it_seller_rating'] = row # Nasze dane z brakami

In [111]:
print(row_comp)

0         50177
1         12428
2          7389
3         15006
4         32975
          ...  
420015      176
420016    34851
420017      983
420018      163
420019      265
Name: it_seller_rating, Length: 420020, dtype: int64


## Algorytm Nearest neighbors

In [64]:
imputer = KNNImputer(n_neighbors=2, weights="uniform")

In [112]:
data2 = imputer.fit_transform(data2)
data2 = pd.DataFrame({'it_seller_rating': data2[:,2]})

In [113]:
print(data2)

       it_seller_rating
0               50483.0
1               12428.0
2                7389.0
3               15006.0
4               32975.0
...                 ...
41997            9306.0
41998            9306.0
41999           15108.0
42000           58407.0
42001           10233.0

[42002 rows x 1 columns]


In [114]:
y_actual = []
y_predicted = []
for i in range(int(n/10)):
    y_actual.append(row_comp.at[i*10])
    y_predicted.append(data2.at[i*10,'it_seller_rating'])

In [122]:
rms = sqrt(mean_squared_error(y_actual,y_predicted ))
scores = [0]*10
scores[9] = rms

In [155]:

for mk in range(9):
    data2 = copy.deepcopy(data)
    data3 = copy.deepcopy(data)
    data2 = data2.head(n)
    data2 = data2[['price', 'it_quantity']]
    row = data3['it_seller_rating']
    row_comp = copy.deepcopy(row)

    for i in range(int(n / 10)):
        row.loc[i * 10 + mk + 1] = nan

    data2['it_seller_rating'] = row  # Nasze dane z brakami
    
    
    data2 = imputer.fit_transform(data2)
    data2 = pd.DataFrame({'it_seller_rating': data2[:, 2]})
    
    y_actual = []
    y_predicted = []
    for i in range(int(n / 10)):
        y_actual.append(row_comp.at[i * 10 + mk + 1])
        y_predicted.append(data2.at[i * 10 + mk + 1, 'it_seller_rating'])
    
    rms = sqrt(mean_squared_error(y_actual, y_predicted))
    scores[mk] = rms

In [156]:
print(scores)

[35834.21492842072, 35575.730588735096, 34316.31153674452, 35906.63482101632, 35868.03616627869, 35019.97981222818, 34834.8771533338, 34159.92270550581, 34236.60675212314, 35935.9751910578]


Dla róznych braków danych wartości średnie mają zbliżone wartośći

In [141]:
data2 = copy.deepcopy(data)
data3 = copy.deepcopy(data)
data4 = copy.deepcopy(data)
data2 = data2.head(n)
data2 = data2[['price']]
row_sel = data3['it_seller_rating']
row_qua = data4['it_quantity']
row_comp_sel = copy.deepcopy(row_sel)
row_comp_qua = copy.deepcopy(row_qua)
for i in range(int(n/10)):
    row_sel.loc[i * 10] = nan
    row_qua.loc[i * 10 + 4] = nan
data2['it_seller_rating'] = row_sel # Nasze dane z brakami
data2['it_quantity'] = row_qua

In [142]:
print(data2)

        price  it_seller_rating  it_quantity
0       59.99               NaN        997.0
1        4.90           12428.0       9288.0
2      109.90            7389.0        895.0
3       18.50           15006.0        971.0
4       19.90           32975.0          NaN
...       ...               ...          ...
41997    2.40            9306.0       1515.0
41998    2.40            9306.0       1515.0
41999    2.00           15108.0          0.0
42000   31.90           58407.0         10.0
42001  429.99           10233.0         66.0

[42002 rows x 3 columns]


In [143]:
data2 = imputer.fit_transform(data2)

In [144]:
data2 = pd.DataFrame({'it_seller_rating': data2[:,1],'it_quantity': data2[:,2]})

In [145]:
print(data2)

       it_seller_rating  it_quantity
0                1543.5        997.0
1               12428.0       9288.0
2                7389.0        895.0
3               15006.0        971.0
4               32975.0         11.5
...                 ...          ...
41997            9306.0       1515.0
41998            9306.0       1515.0
41999           15108.0          0.0
42000           58407.0         10.0
42001           10233.0         66.0

[42002 rows x 2 columns]


In [146]:
    y_actual_sel = []
    y_predicted_sel = []
    y_actual_qua = []
    y_predicted_qua = []
    for i in range(int(n / 10)):
        y_actual_sel.append(row_comp_sel.at[i * 10])
        y_predicted_sel.append(data2.at[i * 10, 'it_seller_rating'])
        y_actual_qua.append(row_comp_sel.at[i * 10])
        y_predicted_qua.append(data2.at[i * 10, 'it_quantity'])
        

In [149]:
rms_sel = sqrt(mean_squared_error(y_actual_sel,y_predicted_sel))
rms_qua = sqrt(mean_squared_error(y_actual_qua,y_predicted_qua))

In [150]:
print(rms_sel,rms_qua)

41497.25540014306 44432.06442848078


Widać że wraz ze zmniejszeniem danych które mamy zwiększyła się niedokładność naszego modelu